# Capstone Project : Mahindra First Choice


### Problem Statement
Mahindra First Choice Services (MFCS) is a company of Mahindra Group and is India’s leading chain of multi-brand car workshops with over 335+ workshops present in 267+ towns & 24 states. It has serviced over 10,50,000 cars. The company aims to establish countrywide network of over 400 workshops by March 2018.
Mahindra would now like to leverage the data that they have and address the key issues they have. Read along to know how you can help them improve their business.
The dataset consist of three aspects:
* Customer data: where the details of the customer like the car owned, state and place of residence, order type, etc are present. Data dimension is of 534000 Customer entries
* Invoice data: where information related to customer visits and transactions are recorded, whether a customer as insurance claims, bifurcation of the amount 
 paid, for what type of service did the customer came for, etc…
* Material Inventory: where information related to what kind of service did the customer took and what kind of material was used to service, Labor information and the cost for the service, Plant and plant name where the customer took the service.


### Objective

### Geolocation Based Customer Analysis:
The idea is to explore how various factors like car make & model, time and type of service etc. vary with location. Since the servicing industry is local in nature, this kind of an analysis could possibly render some really interesting business insights.
Furthermore, this analysis will enable us to formulate more concrete machine learning problems.
From the data at hand it is possible to extract insights about customer behaviour especially the following questions can be addressed

#### Problem Statement-1
Identifying the ownership pattern of cars throughout the country. This also captures the problem wherein information regarding the spending patterns can be identified Expected Business Outcome: Mahindra First Choice Services will be benefited in multiple ways. Knowing the ownership pattern targeted marketing campaigns could be carried out. Knowing the spending patterns services could be suited to the particular spending pattern.

#### Problem Statement-2
Identify the type of order each state receives and present it as an interactive visualization. Expected Business Outcome: This could potentially give information about how Mahindra First Choice needs to be prepared to tackle various seasonal cases

### Market Segmentation: [Solved in this Notebook]
Market segmentation is the process of dividing a market of potential customers into internally homogeneous and mutually heterogeneous groups or segments, based on different characteristics captured in the data. Groups created through such a segmentation exercise many times reveal behavioral patterns which are different from generally accepted segments by the business. The exercise is broadly known as “clustering” and is aimed at finding the consumers who will respond similarly to various stimuli by detecting underlying behavior patterns.
Though clustering falls under a Machine Learning problem category called unsupervised learning, which requires extensive efforts, it is possible to carry out a visual analysis in a relatively short timespan.

#### Problem Statement: 
Customer Lifetime value prediction - Based on Customer segments, predict the revenue that can be extracted from each segment over a life of the car -Regression/Time Series.


#### Why are we doing this
- To Help Mahindra improve its business from analysing the data given like max. car model coming for repair location-wise, top sales garage etc
- To Identify more profitable area or expansion in other regions

#### Stakeholders:
- Marketing Head of Mahindra First [For Marketing Campaign]
- Operational Head/State Head of Mahindra First [For knowing In-Outs operation in Garage]
- Project Manager [i.e. Our Consulting Firm Manager - To which Mahindra First Company are first point of contact with for Analysis]

## Table Of Contents

### 1. <a href="#Importing-Libraries">Libraries Imported </a> 
### 2. <a href="#Import-DataSet">Import DataSet </a>
### 3. <a href="#Clustering-Customer-Based-on-Spending-Pattern">Clustering based on Spending Customer</a>

## Start of Notebook

### Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import plotly
import plotly.graph_objects as go
import re
import pgeocode
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor ,RandomForestRegressor
from xgboost import XGBRegressor 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Ridge,Lasso

In [5]:
 # Some tweaks for displaying large rows
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
# If DataSet lies in another Directory
#os.getcwd()
os.chdir("C:\\Users\\Dashang\\Documents\\GreyAtom_Capstone_MahindraFirst\\") 

## Import DataSet

In [8]:
df = pd.read_csv("invoice_final.csv")   ## Pre-Cleaned DataSet

In [9]:
df["invoice_datetime"] = pd.to_datetime(df["invoice_datetime"],dayfirst=True)

In [10]:
df["invoice_datetime"] = pd.to_datetime(df["invoice_datetime"],dayfirst=True)
df["jobcard_datetime"] = pd.to_datetime(df["jobcard_datetime"],dayfirst=True)
df['Service_Time']     = abs(df['invoice_datetime'] - df['jobcard_datetime'])
df['service_time_hrs'] = df['Service_Time']/np.timedelta64(1,'h')

In [13]:
df.head()

,model,customer_type,customer_no,gate_pass_time,invoice_no,job_card_no,km_reading,labour_total,make,order_type,pin_code,plant_code,plant_name,regn_no,total_amt,user_id,pin_city,pin_district,pin_location,latitude,longitude,state,zones,invoice_datetime,jobcard_datetime,Service_Time,model_type,service_time_hrs
0,spark,retail,67849,0:00:00,7005200002,168303,49317,1203.14,general motors,paid service,400601,bc01,thane,ka19ma1291,4051.95,bc01fs1,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",19.1941,73.0002,maharashtra,west zone,2015-02-01 11:30:00,2014-12-13 14:29:00,49 days 21:01:00,hatchback,1197.016667
1,indica,retail,84419,0:00:00,7005200003,173997,78584,804.26,tata motors,smc value package,400603,bc01,thane,mh43r3046,1001.29,bc01sa2,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.0002,maharashtra,west zone,2015-03-01 10:07:00,2015-02-01 14:12:00,27 days 19:55:00,hatchback,667.916667
2,zen,retail,81055,0:00:00,7005200004,173889,33985,180.19,maruti suzuki,running repairs,400607,bc01,thane,ap09ax0582,233.14,bc01sa2,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh",19.1941,73.0002,maharashtra,west zone,2015-03-01 11:12:00,2015-02-01 11:40:00,27 days 23:32:00,hatchback,671.533333
3,indica,retail,84419,0:00:00,7005200005,174158,78584,0.00,tata motors,smc redemption,400603,bc01,thane,mh43r3046,0.00,bc01sa2,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.0002,maharashtra,west zone,2015-03-01 11:40:00,2015-03-01 10:12:00,0 days 01:28:00,hatchback,1.466667
4,santro xing,retail,18980,0:00:00,7005200006,173860,50057,1202.97,hyundai,paid service,400603,bc01,thane,mh04cd9768,2747.12,bc01sa2,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.0002,maharashtra,west zone,2015-03-01 12:07:00,2015-02-01 10:45:00,28 days 01:22:00,hatchback,673.366667


In [12]:
df.columns

Index(['model', 'customer_type', 'customer_no', 'gate_pass_time', 'invoice_no', 'job_card_no', 'km_reading', 'labour_total', 'make', 'order_type', 'pin_code', 'plant_code', 'plant_name', 'regn_no', 'total_amt', 'user_id', 'pin_city', 'pin_district', 'pin_location', 'latitude', 'longitude', 'state', 'zones', 'invoice_datetime', 'jobcard_datetime', 'Service_Time', 'model_type', 'service_time_hrs'], dtype='object')

In [14]:
df1 = df[['customer_type','customer_no','make','model','order_type','pin_city','pin_location','invoice_no','service_time_hrs','total_amt']]

In [15]:
df1.isnull().sum()

customer_type       0
customer_no         0
make                0
model               0
order_type          0
pin_city            0
pin_location        0
invoice_no          0
service_time_hrs    0
total_amt           0
dtype: int64

In [17]:
main_cluster = df1.groupby(['customer_no','customer_type','make','model','pin_city','order_type']).agg(
                                {
                                 'invoice_no':'nunique',
                                 'total_amt':'sum',
                                 'service_time_hrs':'mean'
                                }
                            )

In [18]:
main_cluster.head()

invoice_no  total_amt  service_time_hrs
customer_no customer_type make           model   pin_city order_type                                              
1           retail        maruti suzuki  kizashi Pune     paid service              1       0.00       5869.083333
                                         omni    Pune     paid service              2       0.00       3370.041667
                                                          running repairs           1       0.00        321.366667
10          retail        general motors optra   Mumbai   paid service              4   10505.72        370.050000
                                                          running repairs           1     599.84          3.066667

In [20]:
main_cluster_reset_index=main_cluster.reset_index()

In [24]:
main_cluster_reset_index.head()

,customer_no,customer_type,make,model,pin_city,order_type,invoice_no,total_amt,service_time_hrs
0,1,retail,maruti suzuki,kizashi,Pune,paid service,1,0.00,5869.083333
1,1,retail,maruti suzuki,omni,Pune,paid service,2,0.00,3370.041667
2,1,retail,maruti suzuki,omni,Pune,running repairs,1,0.00,321.366667
3,10,retail,general motors,optra,Mumbai,paid service,4,10505.72,370.050000
4,10,retail,general motors,optra,Mumbai,running repairs,1,599.84,3.066667


### Clustering Customer Based on Spending Pattern

In [22]:
income_cluster = main_cluster_reset_index[main_cluster_reset_index['total_amt']>0]

In [25]:
print("0.05 quantile :-",income_cluster['service_time_hrs'].quantile(0.05))
print("0.25 quantile :-",income_cluster['service_time_hrs'].quantile(0.25))
print("0.50 quantile :-",income_cluster['service_time_hrs'].quantile(0.50))
print("0.75 quantile :-",income_cluster['service_time_hrs'].quantile(0.75))
print("0.95 quantile :-",income_cluster['service_time_hrs'].quantile(0.95))
print("mimimum servicing time in hrs :-",income_cluster['service_time_hrs'].min())
print("maximum servicing time in hrs :-",income_cluster['service_time_hrs'].max())
print("mean servicing time in hrs :-",income_cluster['service_time_hrs'].mean())

0.05 quantile :- 0.23333333333333334
0.25 quantile :- 3.3833333333333333
0.50 quantile :- 24.1
0.75 quantile :- 452.7825
0.95 quantile :- 3305.8716666666655
mimimum servicing time in hrs :- 0.0
maximum servicing time in hrs :- 18869.066666666666
mean servicing time in hrs :- 557.6629086833316


In [26]:
# Low sales customers (less than 0.50 quantile)
low_sales = income_cluster[(income_cluster['total_amt']< income_cluster['total_amt'].quantile(0.50))]

# Medium sales customers (between 0.50 to 0.75 quantile)
medium_sales = income_cluster[
                            (income_cluster['total_amt'] > income_cluster['total_amt'].quantile(0.50)) & 
                            (income_cluster['total_amt']< income_cluster['total_amt'].quantile(0.75))
                         ]

# Average sales customers (between 0.75 to 0.95 quantile)
avg_sales = income_cluster[
                        (income_cluster['total_amt'] > income_cluster['total_amt'].quantile(0.75)) & 
                        (income_cluster['total_amt'] < income_cluster['total_amt'].quantile(0.95))
                      ]

# High revenue customers (moe than 0.95 quantile)
high_sales = income_cluster[
                        (income_cluster['total_amt'] > income_cluster['total_amt'].quantile(0.95))
                       ]

In [ ]:
low_sales['cluster']='low_sales'
medium_sales['cluster']='medium_sales'
avg_sales['cluster']='avg_sales'
high_sales['cluster']='high_sales'


In [27]:
revenue_clust = pd.concat([low_sales,medium_sales,avg_sales,high_sales],axis=0)

In [ ]:
revenue_clust.groupby(['cluster']).agg({'Total Revenue':'mean','Total Servicing Hrs':'mean','Customer No.':'count'}).sort_values('Total Revenue',ascending=False)